# Deep Learning

# Utilizing Apache Spark and PySpark for Deep Learning in Neuroimaging Studies

## Why Spark?

- **Efficient Large Data Handling:**
   - Spark is designed for fast computation and handling big datasets, essential for neuroimaging data.

- **PySpark - Python Integration:**
   - PySpark provides a Python interface to Spark, making it easier to implement deep learning algorithms using Python libraries.

- **Distributed Computing:**
   - Spark's distributed nature allows for parallel processing, speeding up the deep learning tasks.

- **MLlib for Machine Learning:**
   - Spark's MLlib offers machine learning algorithms optimized for big data, which we can leverage for our deep learning models.

## Advantages:

- Spark handles our complex and large participant data efficiently.
- PySpark allows seamless integration with Python's deep learning libraries.
- Distributed processing capability accelerates our deep learning computations.
- Scalability ensures that our system can grow with our data needs.

In summary, Apache Spark and PySpark provide a robust, scalable, and efficient platform for deep learning in our neuroimaging study, enabling us to process large datasets with speed and ease.


## 1 - Setting up Spark cluster

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("PySpark_VSCode").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/07 06:21:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import models
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


/tmp/ipykernel_314968/3521941988.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-07 06:21:36.103332: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-07 06:21:36.121586: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 06:2

In [4]:
combined_df = pd.read_csv('/home/skander/combined.csv')

In [5]:

# Set 'participant_id' as index
combined_df_with_index = combined_df.set_index('participant_id')

# Extract features for pattern discovery
features = combined_df_with_index  # All columns except 'participant_id'

# Now features DataFrame is ready for unsupervised learning techniques



## 2 - Neuron layer

In [15]:
# Assuming 'features' is your pre-processed dataset ready for training

# Adjust the input dimension to match the number of features in your dataset
input_dim = features.shape[1]  # Number of features

# Define the size of the encoded representations
encoding_dim = 32  # This is a hyperparameter you can tune

# Input layer
input_layer = Input(shape=(input_dim,))

# Encoded layer - where the data is compressed
encoded = Dense(encoding_dim, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)  # Normalize the activations of the previous layer
encoded = Dropout(0.5)(encoded)  # Add dropout to prevent overfitting

# Decoded layer - reconstruction of the input
decoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)  # Normalize the activations of the previous layer
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the model with the Adam optimizer and binary crossentropy loss
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping and reduce learning rate on plateau for better training control
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min', min_lr=0.0001)

# Split the data into training and validation sets for model training and evaluation
from sklearn.model_selection import train_test_split
X_train, X_val, _, _ = train_test_split(features, features, test_size=0.2, random_state=42)

# Train the autoencoder
autoencoder.fit(X_train, X_train,  # Input and output are the same for an autoencoder
                epochs=200,          # Number of epochs
                batch_size=256,     # Batch size
                shuffle=True,       # Shuffle the data
                validation_data=(X_val, X_val),  # Validation data
                callbacks=[early_stopping, reduce_lr])  # Callbacks for monitoring

Epoch 1/200
1/1 [==============================] - 1s 657ms/step - loss: 0.7158 - accuracy: 0.0000e+00 - val_loss: 2.3798 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/200
1/1 [==============================] - 0s 18ms/step - loss: 0.7280 - accuracy: 0.0000e+00 - val_loss: 1.3546 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/200
1/1 [==============================] - 0s 20ms/step - loss: 0.7093 - accuracy: 0.0000e+00 - val_loss: 0.4327 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6677 - accuracy: 0.0213 - val_loss: -0.4817 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6222 - accuracy: 0.0213 - val_loss: -1.3603 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6129 - accuracy: 0.0000e+00 - val_loss: -2.2393 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 7/200
1/1 [=====================

In [7]:
import tensorflow as tf

# Create some tensors and perform operations
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

# Check which device is being used
print('Executing on device:', c.device)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
Executing on device: /job:localhost/replica:0/task:0/device:GPU:0
